Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [28]:
from collections import namedtuple, deque
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [29]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [30]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [31]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 342019.43it/s]


array([[5, 6, 7],
       [8, 0, 2],
       [4, 3, 1]])

In [32]:
def is_goal(state: np.ndarray) -> bool:
    """Check if the state is the goal state."""
    goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.array_equal(state, goal)

## Depth-First Search (DFS)

In [33]:
def dfs_solve(initial_state: np.ndarray) -> list['Action']:
    """Solve the puzzle using Depth-First Search."""
    stack = deque([(initial_state, [])])  # Stack of (state, path)
    visited = set()  # Set to keep track of visited states
    visited.add(initial_state.tobytes())

    while stack:
        current_state, path = stack.pop()

        if is_goal(current_state):
            return path, current_state

        for act in available_actions(current_state):
            new_state = do_action(current_state, act)

            # Avoid revisiting states
            if new_state.tobytes() not in visited:
                visited.add(new_state.tobytes())
                stack.append((new_state, path + [act]))

    return None  # No solution found

In [34]:
print("Initial state:")
print(state)

print("\nSolving with Depth-First Search")
dfs_solution_path, dfs_solution_final_state = dfs_solve(state)
if dfs_solution_path:
    print(f"DFS found a solution with {len(dfs_solution_path)} actions.")
    print(dfs_solution_path)
    print("Final state:")
    print(dfs_solution_final_state)
    
else:
    print("DFS could not find a solution.")

Initial state:
[[5 6 7]
 [8 0 2]
 [4 3 1]]

Solving with Depth-First Search
DFS found a solution with 51760 actions.
[Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2))

In [35]:
from heapq import heappush, heappop

def manhattan_distance(state): # Calculate the Manhattan distance heuristic
    goal_positions = {value: (i // PUZZLE_DIM, i % PUZZLE_DIM) for i, value in enumerate(range(1, PUZZLE_DIM**2))}
    goal_positions[0] = (PUZZLE_DIM - 1, PUZZLE_DIM - 1)
    
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Ignore the blank tile
                gx, gy = goal_positions[value]
                distance += abs(x - gx) + abs(y - gy)
    return distance

In [36]:
def astar_solve(initial_state: np.ndarray) -> list['Action']:
    """Solve the puzzle using the A* algorithm."""
    open_set = []  # Priority queue
    heappush(open_set, (0, initial_state.tobytes(), []))  # (f_score, serialized_state, path)
    visited = set()
    visited.add(initial_state.tobytes())

    while open_set:
        _, serialized_state, path = heappop(open_set)
        current_state = np.frombuffer(serialized_state, dtype=int).reshape(PUZZLE_DIM, PUZZLE_DIM)

        if is_goal(current_state):
            return path, current_state

        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            serialized_new_state = new_state.tobytes()

            if serialized_new_state not in visited:
                visited.add(serialized_new_state)
                g = len(path) + 1
                h = manhattan_distance(new_state)
                f = g + h
                heappush(open_set, (f, serialized_new_state, path + [act]))

    return None  # No solution found


In [37]:
print("Initial state:")
print(state)

print("\nSolving with A*")
astar_solution_path, astar_solution_path_final_state = astar_solve(state)
if astar_solution_path:
    print(f"A* found a solution with {len(astar_solution_path)} actions.")
    print(astar_solution_path)
    print("Final state:")
    print(astar_solution_path_final_state)
    
else:
    print("A* could not find a solution.")

Initial state:
[[5 6 7]
 [8 0 2]
 [4 3 1]]

Solving with A*
A* found a solution with 24 actions.
[Action(pos1=(1, 1), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2))]
Final state:
[[1 2 3]
 [4 5 6]
 [7 8 0]]
